### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [10]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [11]:
X_test.head(3)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,63,F,ATA,140,195,0,Normal,179,N,0.0,Up,0
1,53,M,NAP,145,518,0,Normal,130,N,0.0,Flat,1
2,65,M,ASY,160,0,1,ST,122,N,1.2,Flat,1


In [19]:
import urllib.request
import json      

def get_prediction(Age, Sex, ChestPainType, RestingBP, Cholesterol, FastingBS, RestingECG, MaxHR, ExerciseAngina, Oldpeak, ST_Slope, HeartDisease):
    body = {'Age': Age, 
            'Sex': Sex,
            'ChestPainType': ChestPainType,
            'RestingBP': RestingBP,
            'Cholesterol': Cholesterol,
            'FastingBS': FastingBS,
            'RestingECG': RestingECG,
            'MaxHR': MaxHR,
            'ExerciseAngina': ExerciseAngina,
            'Oldpeak': Oldpeak,
            'ST_Slope': ST_Slope,
            'HeartDisease': HeartDisease,
           } 

    myurl = "http://0.0.0.0:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [20]:
X_test[:1]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,63,F,ATA,140,195,0,Normal,179,N,0.0,Up,0


In [21]:
get_prediction(63, 'F', 'ATA', 140, 195, 0, 'Normal', 179, 'N', 0.0, 'Up', 0)

HTTPError: HTTP Error 500: INTERNAL SERVER ERROR

In [16]:
get_prediction([63], ['F'], ['ATA'], [140], [195], [0], ['Normal'], [179], ['N'], [0.0], ['Up'], [0])

HTTPError: HTTP Error 500: INTERNAL SERVER ERROR

In [14]:
%%time
predictions = X_test[['description', 'company_profile', 'benefits']].iloc[:500].apply(lambda x: get_prediction(x[0], 
                                                                                               x[1], 
                                                                                               x[2]), 1)

CPU times: user 334 ms, sys: 58.3 ms, total: 393 ms
Wall time: 2.38 s


In [15]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.9803161348046525

In [16]:
#X_test[['description', 'company_profile', 'benefits']].head(3)

Давайте подумаем, где наше api может сломаться

In [17]:
get_prediction("", "", None)

0.13518841848060933